In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

KeyboardInterrupt: 

In [ ]:
ratings_df = pd.read_csv("ml-latest-small/ratings.csv")

In [ ]:
ratings_df.head()

In [ ]:
movie_df = pd.read_csv("ml-latest-small/movies.csv")

In [ ]:
movie_df.tail()

In [ ]:
movie_df["movieRow"] = movie_df.index

In [ ]:
movie_df.tail()

In [ ]:
ratings_df = pd.merge(ratings_df,movie_df,on='movieId')

In [9]:
ratings_df.tail()

,userId,movieId,rating,timestamp,title,genres,movieRow
100831,610,160341,2.5,1479545749,Bloodmoon (1997),Action|Thriller,9325
100832,610,160527,4.5,1479544998,Sympathy for the Underdog (1971),Action|Crime|Drama,9330
100833,610,160836,3.0,1493844794,Hazard (2005),Action|Drama|Thriller,9342
100834,610,163937,3.5,1493848789,Blair Witch (2016),Horror|Thriller,9389
100835,610,163981,3.5,1493850155,31 (2016),Horror,9390


In [10]:
ratings_df.head()

,userId,movieId,rating,timestamp,title,genres,movieRow
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0


In [11]:
ratings_df['movieId'] = ratings_df['movieRow']

In [12]:
ratings_df.head()

,userId,movieId,rating,timestamp,title,genres,movieRow
0,1,0,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0
1,5,0,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0
2,7,0,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0
3,15,0,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0
4,17,0,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0


In [13]:
ratings_df.tail()

,userId,movieId,rating,timestamp,title,genres,movieRow
100831,610,9325,2.5,1479545749,Bloodmoon (1997),Action|Thriller,9325
100832,610,9330,4.5,1479544998,Sympathy for the Underdog (1971),Action|Crime|Drama,9330
100833,610,9342,3.0,1493844794,Hazard (2005),Action|Drama|Thriller,9342
100834,610,9389,3.5,1493848789,Blair Witch (2016),Horror|Thriller,9389
100835,610,9390,3.5,1493850155,31 (2016),Horror,9390


In [14]:
ratings_df = ratings_df[['userId','movieId','rating']]

In [15]:
ratings_df.tail()

,userId,movieId,rating
100831,610,9325,2.5
100832,610,9330,4.5
100833,610,9342,3.0
100834,610,9389,3.5
100835,610,9390,3.5


In [16]:
ratings_df.to_csv('ratingProcessed.csv',index=False,header=True,encoding='utf-8')

In [17]:
#创建用户评分矩阵rating和用户行为矩阵record

In [2]:
ratings_df = pd.read_csv('ratingProcessed.csv')

In [3]:
userNo = ratings_df['userId'].max()+1

In [4]:
movieNo = ratings_df['movieId'].max()+1

In [5]:
userNo,movieNo

(611, 9742)

In [6]:
rating = np.zeros((userNo,movieNo))

In [7]:
for index ,row in ratings_df.iterrows():
    rating[int(row['userId']),int(row['movieId'])] = row['rating']

In [8]:
record = rating>0

In [9]:
record = np.array(record,dtype=int)

In [10]:
record

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [11]:
#构建LFM模型

In [12]:
num_f = 10

In [13]:
P_matrix = tf.Variable(tf.random_normal([userNo,num_f],stddev=0.35))

In [14]:
Q_matrix = tf.Variable(tf.random_normal([movieNo,num_f],stddev=0.35))

In [15]:
lamda = 1

In [16]:
C_loss = tf.reduce_sum(((rating-tf.matmul(P_matrix,Q_matrix,transpose_b=True))*record)**2)+\
lamda*(tf.reduce_sum(P_matrix)**2 + tf.reduce_sum(Q_matrix)**2)

In [17]:
optimizer = tf.train.AdamOptimizer(1e-4)
train = optimizer.minimize(C_loss)

In [18]:
#训练模型

In [19]:
tf.summary.scalar('loss',C_loss)

<tf.Tensor 'loss:0' shape=() dtype=string>

In [20]:
summaryMerged = tf.summary.merge_all()

In [21]:
filename='tensorboard'

In [22]:
writer = tf.summary.FileWriter(filename)

In [23]:
sess = tf.Session()

In [24]:
init = tf.global_variables_initializer()

In [25]:
sess.run(init)

In [26]:
for i in range(10000):
    _, movie_summary = sess.run([train,summaryMerged])
    writer.add_summary(movie_summary,i)

In [27]:
Current_P,Current_Q = sess.run([P_matrix,Q_matrix])

In [28]:
predicts = np.dot(Current_P,Current_Q.T)

In [29]:
errors = np.sqrt(np.sum(predicts-rating)**2)/(movieNo*userNo)

In [30]:
errors

2.7128586568292126